In [ ]:
from zenml.materializers.built_in_materializer import BuiltInContainerMaterializer

import polars as pl

artifact_path="gs://zenml-472221-zenml-artifacts/ml_forecast/preprocess_data/training_data/b84e71d0-8024-4924-ac03-42f0ba96a323/a111eb87"

training_dataset = BuiltInContainerMaterializer(
    uri=artifact_path
).load(data_type=dict)


In [8]:
training_dataset["Store_1-Item_A"]

,ds,y
0,2024-01-01,41
15,2024-01-02,17
30,2024-01-03,17
45,2024-01-04,17
60,2024-01-05,19
...,...,...
1005,2024-03-08,27
1020,2024-03-09,42
1035,2024-03-10,90
1050,2024-03-11,39


## test Loading mode from Mlflow Model regsitry

In [4]:
## Loading Model from MLflow Registry
import mlflow

# Set the tracking URI to your MLflow registry
mlflow.set_tracking_uri("https://mlflow.34.40.165.212.nip.io")

# Load model by name and version (replace with your actual model name and version)
model = mlflow.prophet.load_model("models:/prophet_model_Store_1-Item_A/1")


In [9]:
futur = model.make_future_dataframe(periods=30)

predict = model.predict(futur)


In [10]:
predict

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2024-01-01,29.858490,8.569640,46.882874,29.858490,29.858490,-0.061617,-0.061617,-0.061617,-0.061617,-0.061617,-0.061617,0.0,0.0,0.0,28.018710
1,2024-01-02,29.909325,4.131392,42.731726,29.909325,29.909325,-0.190230,-0.190230,-0.190230,-0.190230,-0.190230,-0.190230,0.0,0.0,0.0,24.219676
2,2024-01-03,29.960161,4.196463,39.522662,29.960161,29.960161,-0.270604,-0.270604,-0.270604,-0.270604,-0.270604,-0.270604,0.0,0.0,0.0,21.852810
3,2024-01-04,30.010996,8.779932,45.845221,30.010996,30.010996,-0.063970,-0.063970,-0.063970,-0.063970,-0.063970,-0.063970,0.0,0.0,0.0,28.091183
4,2024-01-05,30.061831,7.988416,43.000329,30.061831,30.061831,-0.115248,-0.115248,-0.115248,-0.115248,-0.115248,-0.115248,0.0,0.0,0.0,26.597271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,2024-04-07,34.792503,31.715232,65.899100,34.792381,34.792629,0.404481,0.404481,0.404481,0.404481,0.404481,0.404481,0.0,0.0,0.0,48.865396
98,2024-04-08,34.843378,16.008284,51.651994,34.843249,34.843511,-0.061617,-0.061617,-0.061617,-0.061617,-0.061617,-0.061617,0.0,0.0,0.0,32.696446
99,2024-04-09,34.894253,10.580351,45.042105,34.894118,34.894393,-0.190230,-0.190230,-0.190230,-0.190230,-0.190230,-0.190230,0.0,0.0,0.0,28.256321
100,2024-04-10,34.945128,7.738216,44.045712,34.944983,34.945276,-0.270604,-0.270604,-0.270604,-0.270604,-0.270604,-0.270604,0.0,0.0,0.0,25.488823


In [12]:
## Save model to bentoml store

import bentoml

# model_uri can be any URI that refers to an MLflow model
# Use local path for demonstration
bentoml.mlflow.import_model(
    "prophet_model_Store_1-Item_A",
    model_uri="models:/prophet_model_Store_1-Item_A/latest"
)

Using the default model signature for MLflow ({'predict': {'batchable': False}}) for model "prophet_model_Store_1-Item_A".
[bentoml] Converting 'prophet_model_Store_1-Item_A' to lowercase: 'prophet_model_store_1-item_a'.


Model(tag="prophet_model_store_1-item_a:ooxr3ieuhgqqsty7", path="/var/folders/7m/f7p55f_n32s_qv2f0fpnwyrh0000gn/T/bentoml-model-prophet_model_store_1-item_a-4n5bzwni")

In [13]:
prophet_model = bentoml.mlflow.load_model("prophet_model_Store_1-Item_A:latest")

[bentoml] Converting 'prophet_model_Store_1-Item_A' to lowercase: 'prophet_model_store_1-item_a'.


In [16]:
type(prophet_model.model)

AttributeError: 'PyFuncModel' object has no attribute 'model'

In [19]:
futur['y'] = 1
prophet_model.predict(futur)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2024-01-01,29.858490,9.822998,45.849648,29.858490,29.858490,-0.061617,-0.061617,-0.061617,-0.061617,-0.061617,-0.061617,0.0,0.0,0.0,28.018710
1,2024-01-02,29.909325,6.478804,42.064417,29.909325,29.909325,-0.190230,-0.190230,-0.190230,-0.190230,-0.190230,-0.190230,0.0,0.0,0.0,24.219676
2,2024-01-03,29.960161,4.083992,38.968438,29.960161,29.960161,-0.270604,-0.270604,-0.270604,-0.270604,-0.270604,-0.270604,0.0,0.0,0.0,21.852810
3,2024-01-04,30.010996,10.350653,44.631018,30.010996,30.010996,-0.063970,-0.063970,-0.063970,-0.063970,-0.063970,-0.063970,0.0,0.0,0.0,28.091183
4,2024-01-05,30.061831,8.271286,43.447025,30.061831,30.061831,-0.115248,-0.115248,-0.115248,-0.115248,-0.115248,-0.115248,0.0,0.0,0.0,26.597271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,2024-04-07,34.792503,32.031070,66.515641,34.792373,34.792635,0.404481,0.404481,0.404481,0.404481,0.404481,0.404481,0.0,0.0,0.0,48.865396
98,2024-04-08,34.843378,15.522295,50.433102,34.843241,34.843519,-0.061617,-0.061617,-0.061617,-0.061617,-0.061617,-0.061617,0.0,0.0,0.0,32.696446
99,2024-04-09,34.894253,10.553862,46.763305,34.894106,34.894403,-0.190230,-0.190230,-0.190230,-0.190230,-0.190230,-0.190230,0.0,0.0,0.0,28.256321
100,2024-04-10,34.945128,7.094282,42.489941,34.944973,34.945285,-0.270604,-0.270604,-0.270604,-0.270604,-0.270604,-0.270604,0.0,0.0,0.0,25.488823


In [ ]:
futur['constant'] = 1